# Tracé des résultats élections législatives 2022 ville par ville

## Imports

In [ ]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import requests

## 1er tour

### Acquisition des données

In [ ]:
def get_data(url):
    """Fonction qui télécharge le jeu de données, le sauve sur le disque, et renvoie une pandas.DataFrame avec les données chargées.
    Prend plus de 30s à lire les données, soyez patients"""
    
    print('Téléchargement du fichier de données')
    response = requests.get(url)

    data_file = 'data.xlsx'

    with open(data_file, 'wb') as file:
        file.write(response.content)
    print('Données écrites sur le disque')
    print('Création de la DataFrame, soyez patient…')
    data =  pd.read_excel(data_file)
    print("Ajout des noms de colonnes sur tous les candidats")
    # adding column names to unnamed columns
    unnamed_cols = [col for col in data.columns if col.startswith('Unnamed')]
    missing_column_names = data.columns[21:29].tolist() # taking col names from cols 21 to 29. Changes from 1er tour to 2nd tour :(
    # dict with all the unnamed columns names : proper name
    renamming_cols_dict = dict(
    zip(unnamed_cols, missing_column_names*(len(unnamed_cols)//8))
    )
    # updating column names with the dict
    data = data.rename(columns=renamming_cols_dict)
    print("Données prêtes")
    return data

In [ ]:
url = "https://www.data.gouv.fr/fr/datasets/r/3cf49bf4-c171-4328-b002-fea995cb85d8"

In [ ]:
data = get_data(url)

In [ ]:
data.head()

### Élaboration du graphe par ville

In [ ]:
def plot_city(city_name):

    commune = 'Libellé de la commune'
    département = 'Libellé du département'
    
    # on prend les lignes correspondant à la ville city_name
    df = data[data[commune] == city_name].dropna(axis=1)
    
    # on stocke le nom du département
    dep_name = df[département].iloc[0]
    # on garde les numéros de circonscription
    circo_codes = df['Code de la circonscription'].to_list()
    
    # on se concentre maintenant juste sur les colonnes avec les résultats des candidats (les dernières à partir de la 21
    city = df.iloc[:,21:]
    
    # pour les grandes villes qui ont plusieurs circonscriptions, on fait une liste de dataframes
    circos = [pd.DataFrame(city.iloc[i, :]).T for i in range(0, city.shape[0])]

    circos_clean = []
    # là on doit pour chaque circo remodeler les données : on a une seule ligne avec les différents candidats et leurs scores en colonnes
    # mais on veut empiler les données des candidats les uns sous les autres. On découpe les lignes toutes les 8 colonnes et on empile
    # avec un slicing glissant et un pd.concat()
    for circo in circos:
        frames = []
        for i in range(0, circo.shape[1], 8):
            frames.append(circo.iloc[:, i: i+8])
        circo_df = pd.concat(frames)
        circos_clean.append(circo_df)

    # on prépare un graph avec subplots, une ligne par circonscription, la taille globale dépend du nbre de circonscription
    fig, axs = plt.subplots(len(circos), 1, figsize=(16,10*len(circos)))
    
    # pour traiter les villes avec une seule circonspection comme celles avec plusieurs
    if len(circos_clean) == 1:
        axs = [axs,]
    metric1, metric2 = data.columns[27:29].tolist() # on récupère les métrics % voix/ins et % voix/exp. 
    # On prend via les noms de colonnes 27 et 28 parce que bien sûr entre le 1er tour et le 2nd tour ils ont changé le nom des colonnes
    
    # on fait le même plot pour chaque circonscription
    for num, circo, ax in zip(circo_codes, circos_clean, axs):
        circo[["Nom", "Prénom", "Nuance", metric1, metric2]].sort_values(
            metric1, 
            ascending=False
        ).plot(
            x='Nuance',
            kind='bar',
            title=f"{city_name} circonspection n°{num} du département {dep_name}",
            ax=ax
        );
        

In [ ]:
plot_city('Toulon')

In [ ]:
plot_city('Lyon')

## Second Tour

In [ ]:
url_second_tour = "https://www.data.gouv.fr/fr/datasets/r/a3c524fd-eed6-4be2-99ca-bc3920824953"

In [ ]:
data = get_data(url_second_tour)

In [ ]:
data.head()

In [ ]:
plot_city('Chuelles')

In [ ]:
plot_city('Marseille')

## Nettoyage (suppression du fichier excel)

In [ ]:
os.remove("data.xlsx")